#### Dependencies

In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1)
device = 'cuda' if torch.cuda.is_available() else 'cpu' 
print(device)

cuda


#### Hyperparameters

In [2]:
batch_size = 64
block_size = 265
max_iters = 5000
eval_intervals = 500
learning_rate = 3e-3
eval_iters = 200
n_embd = 384 
n_head = 6
n_layers = 6
dropout = 0.2 

#### Dataset

In [3]:
carm = open(r'./../Datasets/Text/Carmilla.txt')
dor = open(r'./../Datasets/Text/DorianGray.txt')
drac = open(r'./../Datasets/Text/Dracula.txt')
edgar = open(r'./../Datasets/Text/EdgarAllanPoe.txt')
fran = open(r'./../Datasets/Text/Frankenstein.txt')
screw = open(r'./../Datasets/Text/TurnOfScrew.txt')
vamp = open(r'./../Datasets/Text/Vampyre.txt')
wuth = open(r'./../Datasets/Text/WutheringHeights.txt')
yWal = open(r'./../Datasets/Text/YellowWalpaper.txt')
text = ""
text += carm.read() + dor.read() + dor.read() + drac.read() + edgar.read() + fran.read() + screw.read() + vamp.read() + wuth.read() + yWal.read()

In [4]:
chars = sorted(list(set(text)))
vocab_size = len(chars)

In [5]:
stoi = {ch : i for i,ch in enumerate(chars)}
itos = {i : ch for i,ch in enumerate(chars)}

def encode(chars):
    return [stoi[ch] for ch in chars]
def decode(tokens):
    return ''.join([itos[i] for i in tokens])

In [6]:
#Dataset
data = torch.tensor(encode(text), dtype =torch.long)

In [7]:
#Split
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

In [8]:
#generate small batches of x and y
def get_batch(split):
    data = train_data if split == 'train' else val_data
    #selecting random subBatches
    ix = torch.randint(len(data) - block_size, (batch_size,))
    #torch.randint(highvalue, tensorShape)
    x = torch.stack([data[i: i+block_size] for i in ix])
    #Converts List of tensors into 2d tensors
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

In [9]:
@torch.no_grad()
def estimate_loss(model):
    out = {}
    model.eval()
    for split in ['train','val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X,Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

#### Model v1 (Single Head Attention)

In [10]:
class Head(nn.Module):
    def __init__(self,head_size):
        super().__init__()
        self.key = nn.Linear(n_embd,head_size, bias = False)
        self.query = nn.Linear(n_embd,head_size, bias = False)
        self.value = nn.Linear(n_embd,head_size, bias = False)
        #not updated by the optimizer
        #typically used to store non-learnable data that should be saved and loaded with the model
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)
        q = self.query(x)
        v = self.value(x)
        dk = k.shape[-1] ** (-0.5)
        wei = q @ k.transpose(-2,-1) * dk
        wei = wei.masked_fill(self.tril[:T,:T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)

        out = wei @ v
        return out


In [11]:
class nanoGPT(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size,n_embd)
        self.positional_embedding_table = nn.Embedding(block_size,n_embd)
        self.sa_head = Head(n_embd)
        self.lm_head = nn.Linear(n_embd,vocab_size)

    def forward(self, idx, target = None):
        B,T = idx.shape
        tok_emb = self.token_embedding_table(idx)
        pos_emb = self.positional_embedding_table(torch.arange(T,device=device))
        x = tok_emb + pos_emb
        x = self.sa_head(x)
        logits = self.lm_head(x)

        if target is None:
            loss = None
        else:
            B,T,C = logits.shape
            logits = logits.view(B*T,C)
            targets = target.view(B*T)
            loss = F.cross_entropy(logits,targets)

        return logits,loss
    
    def generate(self,idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = idx[:,-block_size:]
            logits, loss = self(idx_cond)
            logits = logits[:,-1,:]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx,idx_next),dim=1)
        return idx

In [12]:
torch.arange(10,device=device)

tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], device='cuda:0')

In [13]:
model = nanoGPT()
model = model.to(device)

In [14]:
idx = torch.zeros((1,1), dtype=torch.long,device=device)
print("Input:",idx)
print("Output:",model.generate(idx, max_new_tokens=100))

Input: tensor([[0]], device='cuda:0')
Output: tensor([[  0, 105,  22, 105,  11,  87,  43, 114,  25,  51,  98,  72,  46,  32,
           6,  88,  72,  90, 115,  92, 112,  59,  83,  83,  75, 107,   8,  10,
          66,  66,  62,  26, 111,  94,  58,  78,  30,  27,  48, 113,  82, 101,
          62,  46,  23, 100,  83, 101,  98,  36,  53,  69,  90, 115, 101,   4,
          18, 101,  37,   5,  65,  99, 111,  20,  77,  31,  73,  23,  67,  94,
          83,  88,  78,  49,  26,  39, 114,  55,  83,  13,  65,  12,  24,  53,
          51,  35,  94,  29,  55,  52,  89, 103,  87,  36,  42,  38,  95,  30,
           7,  65,  95]], device='cuda:0')


In [15]:
#Training Loop
optimizer = torch.optim.AdamW(model.parameters(), learning_rate)
for steps in range(10000):
    xb, yb = get_batch('train')
    logits, loss = model(xb,yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.298800468444824


In [16]:
idx = torch.zeros((1,1), dtype=torch.long).to(device)
print(decode(model.generate(idx, max_new_tokens=100)[0].tolist()))


          tihnin!Geubabethart seedp ofrane—s
Tho ut owok wil soxs wrsprimantppenegeno the; brsesedey


#### Model v2 (MultiHead Attention)

In [17]:
class MultiHead(nn.Module):
    def __init__(self,num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])

    def forward(self,x):
        return torch.cat([h(x) for h in self.heads], dim=-1)


In [18]:
class nanoGPT(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size,n_embd)
        self.positional_embedding_table = nn.Embedding(block_size,n_embd)
        self.sa_heads = MultiHead(n_head,n_embd//n_head)    #6 heads of 8d self attension
        self.lm_head = nn.Linear(n_embd,vocab_size)

    def forward(self, idx, target = None):
        B,T = idx.shape
        tok_emb = self.token_embedding_table(idx)
        pos_emb = self.positional_embedding_table(torch.arange(T,device=device))
        x = tok_emb + pos_emb
        x = self.sa_heads(x)
        logits = self.lm_head(x)

        if target is None:
            loss = None
        else:
            B,T,C = logits.shape
            logits = logits.view(B*T,C)
            targets = target.view(B*T)
            loss = F.cross_entropy(logits,targets)

        return logits,loss
    
    def generate(self,idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = idx[:,-block_size:]
            logits, loss = self(idx_cond)
            logits = logits[:,-1,:]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx,idx_next),dim=1)
        return idx

In [19]:
model = nanoGPT()
model = model.to(device)
idx = torch.zeros((1,1), dtype=torch.long,device=device)
print("Input:",idx)
print("Output:",model.generate(idx, max_new_tokens=100))
#Training Loop
optimizer = torch.optim.AdamW(model.parameters(), learning_rate)
for steps in range(10000):
    xb, yb = get_batch('train')
    logits, loss = model(xb,yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())
idx = torch.zeros((1,1), dtype=torch.long).to(device)
print(decode(model.generate(idx, max_new_tokens=100)[0].tolist()))

Input: tensor([[0]], device='cuda:0')
Output: tensor([[  0,  44,  20,  24, 101,  95,   9,  48,  57,  60,  95,  36,   0, 111,
          43,  35,  95,  68,  52, 115,  38,  18,  14, 101,  28,  77,  44,  39,
          59,  69,  47,  57,  65,   2,  37,  64,  36,  26,  57, 107,  10,  15,
           0,  28,  93,  97,  71,  75,   9,  26,  64,  95,  73, 112,  37,  62,
         114,   7,  11,  21,  70,  84,  10,  16,  10,  75,  83,  97,  35,  53,
          47,   0,  26,  42,  62,  40,  75,  27, 103,  95,  70,  14,  72, 109,
           2,  10,  36,  74,  94,  36,  55,  10,  11,  41,  59,  95,  38,  61,
           7,  19,  65]], device='cuda:0')
1.9322537183761597

c ansheambe to fixplaty at eof the hay a self ever ase
prowethe
“Project Gutenbe for beeneensichsue 


#### Model v3 (Multihead + FeedForward)

In [20]:
class FeedForward(nn.Module):
    def __init__(self,n_embd):
        super().__init__()
        self.net = nn.Sequential(
        nn.Linear(n_embd, n_embd),
        nn.ReLU()
        )
    
    def forward(self,x):
        return self.net(x)

In [21]:
class nanoGPT(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size,n_embd)
        self.positional_embedding_table = nn.Embedding(block_size,n_embd)
        self.sa_heads = MultiHead(n_head,n_embd//n_head)    #6 heads of 8d self attension
        self.ffwd = FeedForward(n_embd)
        self.lm_head = nn.Linear(n_embd,vocab_size)

    def forward(self, idx, target = None):
        B,T = idx.shape
        tok_emb = self.token_embedding_table(idx)
        pos_emb = self.positional_embedding_table(torch.arange(T,device=device))
        x = tok_emb + pos_emb
        x = self.sa_heads(x)
        x = self.ffwd(x)
        logits = self.lm_head(x)

        if target is None:
            loss = None
        else:
            B,T,C = logits.shape
            logits = logits.view(B*T,C)
            targets = target.view(B*T)
            loss = F.cross_entropy(logits,targets)

        return logits,loss
    
    def generate(self,idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = idx[:,-block_size:]
            logits, loss = self(idx_cond)
            logits = logits[:,-1,:]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx,idx_next),dim=1)
        return idx

In [22]:
model = nanoGPT()
model = model.to(device)
idx = torch.zeros((1,1), dtype=torch.long,device=device)
print("Input:",idx)
print("Output:",model.generate(idx, max_new_tokens=100))
#Training Loop
optimizer = torch.optim.AdamW(model.parameters(), learning_rate)
for steps in range(10000):
    xb, yb = get_batch('train')
    logits, loss = model(xb,yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())
idx = torch.zeros((1,1), dtype=torch.long).to(device)
print(decode(model.generate(idx, max_new_tokens=100)[0].tolist()))

Input: tensor([[0]], device='cuda:0')
Output: tensor([[  0,  57,  82,  76,  60,   0,  20,   1,  16,  26, 101,  17,  74, 113,
          61,  83,  82,  55,  19,  70,  74,  34,  27,  71,  41,  62,  99,  30,
          27,  31,   8,   7,  92,  53,  24,  19,  32,  50,  34,  56,  34,  62,
          47,  65,  72,  73,  94,  96,  99,  63, 111,  35, 109,  68,  58, 105,
          12,  45,  65,  23,  20,   4,  65,  30,  80,  41,  97, 104,  70,  15,
          11,   7,  80,  87,  83,  16,  78,  89,  33,  85, 109,  84,  59,   6,
          96,   8, 111,  57, 101,  13,  89,  55,  54, 104,  66,  98,  39,  69,
         114,  95,  24]], device='cuda:0')
1.5401018857955933

san-bruggest shay the gold answernary.
    Oh, vision sphavemen thimself only this ded other perhow 


#### Model v4 (Transformer)

In [32]:
class Head(nn.Module):
    def __init__(self,head_size):
        super().__init__()
        self.key = nn.Linear(n_embd,head_size, bias = False)
        self.query = nn.Linear(n_embd,head_size, bias = False)
        self.value = nn.Linear(n_embd,head_size, bias = False)
        #not updated by the optimizer
        #typically used to store non-learnable data that should be saved and loaded with the model
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)
    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)
        q = self.query(x)
        v = self.value(x)
        dk = k.shape[-1] ** (-0.5)
        wei = q @ k.transpose(-2,-1) * dk
        wei = wei.masked_fill(self.tril[:T,:T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)

        wei = self.dropout(wei)

        out = wei @ v
        return out


In [43]:
class MultiHead(nn.Module):
    def __init__(self,num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.projection = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)
    def forward(self,x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.projection(out)
        out = self.dropout(out)
        return out


In [44]:
class FeedForward(nn.Module):
    def __init__(self,n_embd):
        super().__init__()
        self.net = nn.Sequential(
        nn.Linear(n_embd, 4 * n_embd),
        nn.ReLU(),
        nn.Linear(4 * n_embd, n_embd),  #Projection Layer
        nn.Dropout(dropout),
        )
    
    def forward(self,x):
        return self.net(x)

In [ ]:
class Block(nn.Module):
    #Decoder Transformer Block (lacks cross attention but has mask)
    #Intersparse
    #Communication(MultiHead Attention) and Computation(FeedForward)
    #PreLayerNorm Formulation
    def __init__(self,n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHead(n_head, head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)
    def forward(self,x):
        x = self.ln1(x)
        x = x + self.sa(x)      #Residual Connections/ Skip Connection
        x = self.ln2(x)
        x = x + self.ffwd(x)
        return x 

In [46]:
class nanoGPT(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size,n_embd)
        self.positional_embedding_table = nn.Embedding(block_size,n_embd)
        self.blocks = nn.Sequential(
            *[Block(n_embd, n_head = n_head) for _ in range(n_layers)]
        )
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd,vocab_size)

    def forward(self, idx, target = None):
        B,T = idx.shape
        tok_emb = self.token_embedding_table(idx)
        pos_emb = self.positional_embedding_table(torch.arange(T,device=device))
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)

        if target is None:
            loss = None
        else:
            B,T,C = logits.shape
            logits = logits.view(B*T,C)
            targets = target.view(B*T)
            loss = F.cross_entropy(logits,targets)

        return logits,loss
    
    def generate(self,idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = idx[:,-block_size:]
            logits, loss = self(idx_cond)
            logits = logits[:,-1,:]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx,idx_next),dim=1)
        return idx

In [ ]:
model = nanoGPT()
model = model.to(device)
idx = torch.zeros((1,1), dtype=torch.long,device=device)
print("Input:",idx)
print("Output:",model.generate(idx, max_new_tokens=100))
#Training Loop
optimizer = torch.optim.AdamW(model.parameters(), learning_rate)
for steps in range(10000):
    xb, yb = get_batch('train')
    logits, loss = model(xb,yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())
idx = torch.zeros((1,1), dtype=torch.long).to(device)
print(decode(model.generate(idx, max_new_tokens=100)[0].tolist()))

Input: tensor([[0]], device='cuda:0')
Output: tensor([[  0,  78, 112,  26,  82,  21,  86,  27, 103,  35,  25,   3,   7, 114,
          46,  42,  58,  33,  20,  37,  52, 107,  43,   3,  43,  27,  27,  25,
          51,  25,   1,  87,  74,  38,  76,   2,  88,  25,  14, 102,  92,  89,
          27,  29,   5,  74,   8,  34,  83,  15,  68, 102,   4,  91, 115,  25,
          26, 102,  95,  78, 103,  20,  25,  98,  81,  81,   2,  26,  68,  18,
          36, 113,  45, 102,  70,  20, 104,  74,  19,  64,  65,  54,  72,  42,
         102,  43, 113,  42,  81,  21,  37,  89,   7,  34,  65, 108, 113,  90,
         108,  45,  91]], device='cuda:0')
